In [4]:
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

import json 
import os
import io
import collections
import math
import random
import zipfile

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from pprint import pprint
from shutil import copyfile, move

from PIL import Image

import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
#     config.gpu_options.per_process_gpu_memory_fraction = .7
    return tf.Session(config=config)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


In [5]:
WIDTH = 224
HEIGHT = 224
as_grey = False
batch_size = 16
test_batch_size = 16
epochs = 500

In [267]:
# junk model for display in presentation
n_model = keras.models.load_model(os.getcwd() + '/models/cloud_July 04, 2018.hdf5')
# n_model.summary()

In [281]:
# best non pretrained resnet for indiviuals
# n_model = keras.models.load_model(os.getcwd() + '/models/cloud_custom_August 19, 2018.hdf5')

# best pretrained resnet for indiviuals
# n_model = keras.models.load_model(os.getcwd() + '/models/cloud_pretrained_August 19, 2018.hdf5')

# adapted species resnet for indiviuals
# n_model = keras.models.load_model('D:/Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-184-0.58.hdf5')
# second adapted resnet worse
# n_model2 = keras.models.load_model('D:/Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-416-0.96.hdf5')`

# best pretrained resnet for species
n_model = keras.models.load_model('D:/Species_Snapshots/Wednesday_9_19_2018_inital/weights-improvement-24-0.95.hdf5')

In [31]:
n_model.summary()

In [9]:
from vis.visualization import visualize_cam
from vis.utils import utils

# test_path = os.getcwd() + '/data/Clouded_leopard_ID/test_cleaned_cropped'
test_path = 'D:/Species/test_cropped'
# test_path = 'U:/PycharmProjects/AnimalBiometrics/data/Clouded_leopard_ID/Individual/test_cleaned'

test_datagen = ImageDataGenerator(
        rescale=1./255)

data_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(HEIGHT, WIDTH),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

data_list = []
batch_index = 0

# while batch_index <= data_generator.batch_index:
#     data = data_generator.next()
#     data_list.append(data[0][0])
#     batch_index += 1
    
# data_array = np.asarray(data_list)
# data_array_label = np.asarray(data_generator.classes)

Found 102288 images belonging to 87 classes.


In [11]:
class_dict = {v: k for k, v in data_generator.class_indices.items()}

In [20]:
# n = 102288
# for _ in range(n):
#     data = data_generator.next()
#     data_list.append(data[0][0])
    
# data_array = np.asarray(data_list)
# # data_array_label = np.asarray(data_generator.classes[:data_generator.batch_index])

In [9]:
# load existing prediction
prediction_heat = pd.read_csv('./Prediction_Species_all.csv', usecols=[1]).values.flatten()
data_array_label = data_generator.classes

In [284]:
# make new prediction
prediction_heat_all = n_model.predict_generator(data_generator, verbose=1)
prediction_heat = np.argmax(prediction_heat_all,axis=-1)
data_array_label = data_generator.classes

114/114 [==============================] - 8s 67ms/step


In [66]:
# save predictions
pd.DataFrame(np.concatenate([np.array(data_generator.filenames)[:,np.newaxis], prediction_heat[:,np.newaxis],data_array_label[:,np.newaxis]],axis=1), columns=['Image_path','Prediction', 'True']).to_csv('Prediction_Species_all.csv', index=False)

In [36]:
eval_all = n_model.evaluate_generator(data_generator, verbose=1)

114/114 [==============================] - 48s 423ms/step


In [285]:
correct_pred = (prediction_heat == data_array_label).nonzero()[0]
wrong_pred = (prediction_heat != data_array_label).nonzero()[0]

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(data_array_label, prediction_heat)

0.9488111997497263

In [11]:
import scipy.ndimage as ndimage
from vis.visualization import visualize_saliency
import matplotlib.gridspec as gridspec

def compute_saliency_map(model, array, target_class, layer_idx=-1):
    grads = visualize_saliency(model, layer_idx, filter_indices=target_class, seed_input=array)
#     print(grads.shape)
    smoothe = ndimage.gaussian_filter(grads, sigma=5) 
    return smoothe

def render_img_on_grid(img, pos, grid):
    ax = plt.subplot(grid[pos])
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    return ax

def show_side_by_side(img, saliency_map, pred, true, i=None, save_path='C:\\Users\\fotto\\Dropbox'):
    fig = plt.figure(figsize=(20,10))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    grid = gridspec.GridSpec(1, 2, wspace=0.)
    render_img_on_grid(img, 0, grid)
    text = 'True: ' + true
    plt.title(text, size="xx-large")
    ax = render_img_on_grid(img, 1, grid)
    ax.imshow(saliency_map, alpha=.7)
    text = 'Pred: ' + pred
    ax.set_title(text, size="xx-large")
    if i is not None and save_path:
        root = save_path + '{}_{}_1'.format(true, pred)
        file_name = 'Activation_' + str(i) + '.jpg'
        path = os.path.join(root, file_name)

        if os.path.isdir(root):
            while os.path.isfile(path):
                root = root[:-1] + str(int(root[-1])+1)
                path = os.path.join(root, file_name)
        if not os.path.isdir(root):
            os.mkdir(root)
        plt.savefig(path)
        plt.close(fig)
    else:
        plt.show()
        plt.close(fig)

In [16]:
def print_comparison_v2(indices, images, predictions, labels, trained_model, activation_layer='predictions'):
    
#     trained_model.layers[utils.find_layer_idx(trained_model, "predictions")].activation = keras.activations.linear
    
    for i, img_number in enumerate(indices):
        saliency_map = compute_saliency_map(trained_model, array=images[img_number],
                                            target_class=[labels[img_number]], layer_idx=utils.find_layer_idx(trained_model, activation_layer))
        show_side_by_side(images[img_number], saliency_map, class_dict[predictions[img_number]], class_dict[labels[img_number]])

In [29]:
print_comparison_v2(wrong_pred, data_array, prediction_heat, data_array_label, n_model, None)

In [30]:
print_comparison_v2(correct_pred, data_array, prediction_heat, data_array_label, n_model, None)

In [6]:
def get_data_dist(img_path="U:\\PycharmProjects\\AnimalBiometrics\\data\\Clouded_leopard_ID\\"):
    res = {}
    for subdir, dirs, files in os.walk(img_path):
#         if 'coarse' not in subdir.lower():
#             continue
        if dirs == []:
            idx = subdir.rfind("\\")+1
            res[subdir[idx:]]=len(files)    
            
    return res

In [7]:
dist_res = get_data_dist(img_path="D:\\Species\\train_cropped")
# dist_res = get_data_dist('U:/PycharmProjects/AnimalBiometrics/data/Clouded_leopard_ID/Individual/train_cleaned_copy')
res = collections.OrderedDict(sorted(dist_res.items()))

In [287]:
correct_pred = (prediction_heat == data_generator.classes).nonzero()[0]
wrong_pred = (prediction_heat != data_generator.classes).nonzero()[0]

In [15]:
def compare_per_class(n, data_generator, model, predictions, correct_idx, wrong_idx, res_dict, class_dict, layers=[]):

    for animal in list(res_dict.keys()):
        names = np.array(data_generator.filenames)
        idx = np.flatnonzero(np.core.defchararray.find(names,animal)!=-1)
        idx_correct = np.intersect1d(idx, correct_idx)
        idx_wrong = np.intersect1d(idx, wrong_idx)

        if len(idx_correct) > n:
            idx_correct = np.random.choice(idx_correct, n)

        if len(idx_wrong) > n:
            idx_wrong = np.random.choice(idx_wrong, n)

#         if len(idx) > n:
#             idx = np.random.choice(idx, n)
        
#         print('Examples for {}'.format(animal))
#         for elem in idx:
#             data_generator.batch_index = elem
#             image = data_generator.next()[0][0]
#             prediction = np.argmax(model.predict(np.expand_dims(image, axis=0)))
#             target_class = data_generator.classes[elem]
            
#             saliency_map = compute_saliency_map(model, array=image, target_class=[target_class],
# #                                                 layer_idx=utils.find_layer_idx(model, 'predictions')
#                                                )

#             show_side_by_side(image, saliency_map, class_dict[prediction], class_dict[target_class])

        print('Wrong cases for {}'.format(animal))
        for elem in idx_wrong:
            data_generator.batch_index = elem
            image = data_generator.next()[0][0]
            target_class = data_generator.classes[elem]
            
            if not layers:
                saliency_map = compute_saliency_map(model, array=image, target_class=[target_class])
                show_side_by_side(image, saliency_map, class_dict[predictions[elem]], class_dict[target_class])
            else:
                for i in layers:
                    saliency_map = compute_saliency_map(model, array=image, target_class=[target_class],
                                                    layer_idx=utils.find_layer_idx(model, 'activation_{}'.format(i))
                                                   )
                    show_side_by_side(image, saliency_map, class_dict[predictions[elem]], class_dict[target_class], i)

        print('Correct cases for {}'.format(animal))
        for elem in idx_correct:
            data_generator.batch_index = elem
            image = data_generator.next()[0][0]
            target_class = data_generator.classes[elem]
            
            if not layers:
                saliency_map = compute_saliency_map(model, array=image, target_class=[target_class])
                show_side_by_side(image, saliency_map, class_dict[predictions[elem]], class_dict[target_class])
            else:
                for i in layers:
                    saliency_map = compute_saliency_map(model, array=image, target_class=[target_class],
                                                        layer_idx=utils.find_layer_idx(model, 'activation_{}'.format(i))
                                                       )
                    show_side_by_side(image, saliency_map, class_dict[predictions[elem]], class_dict[target_class], i)

In [ ]:
data_generator.batch_index = 1
compare_per_class(2, data_generator, n_model, prediction_heat, correct_pred, wrong_pred res, class_dict, layers=range(50, 98))

Wrong cases for Asian small-clawed otter
Correct cases for Asian small-clawed otter
Wrong cases for Banded civet
Correct cases for Banded civet
Wrong cases for Banded linsang
Correct cases for Banded linsang
Wrong cases for Banteng
Correct cases for Banteng
Wrong cases for Bay cat
Correct cases for Bay cat
Wrong cases for Bearded pig
Correct cases for Bearded pig
Wrong cases for Binturong
Correct cases for Binturong
Wrong cases for Bird
Correct cases for Bird
Wrong cases for Black-capped babbler


In [15]:
def simplify_labels(true, pred, count_dict):
    true_copy = np.copy(true)
    pred_copy = np.copy(pred)
    class_dict= {}
        
    for animal, count in count_dict.items():
            
        label_idx = (true_copy == animal).nonzero()[0]
        pred_idx = (pred_copy == animal).nonzero()[0]

        if animal in ['Team', 'Ghost', 'Unknown', 'Human', 'Poacher', 'SFD Staff', 'Unsure', 'Vehicle', 'Encroacher']:
            top_level = 'No Animal'
            key = 0
        elif count >= 1000:
            top_level = 'Normal > 1000'
            key = 1
        elif count >= 500:
            top_level = 'Scarce > 500'
            key = 2
        elif count >= 200:
            top_level = 'Rare > 200'
            key = 3
        elif count >= 50:
            top_level = 'Very Rare > 50'
            key = 4
        elif count < 50:
            top_level = 'Extremely Rare < 50'
            key = 5
            
        if key not in class_dict.keys():
            class_dict[key] = top_level
        
        true_copy[label_idx] = top_level
        pred_copy[pred_idx] = top_level
    return true_copy, pred_copy, class_dict

In [18]:
from sklearn.metrics import confusion_matrix, classification_report
import itertools

In [294]:
def plot_confusion_matrix(cm, 
                          class_dict,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          selection=None,
                          font_size=30):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm[np.isnan(cm)] = 0.
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        
    fig = plt.figure(1, figsize=(15,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=font_size)
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=font_size)
    ordered_classes = collections.OrderedDict(sorted(class_dict.items()))
    tick_marks = np.arange(len(ordered_classes.keys()))
#     ticks = [elem if not elem == 'Extremely Rare < 50' else 'Extremely\n Rare < 50' for elem in ordered_classes.values()]
    plt.yticks(tick_marks, ticks, fontsize=font_size)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 fontsize=font_size-5)

    plt.ylabel('True label', size="large")
#     plt.xlabel('Predicted label', size="large")
    plt.tight_layout()
    
    if selection is not None:
        plt.figure(2, figsize=(20,15))
        for i, elem in enumerate(ordered_classes.values()):
            ax = plt.subplot(1,len(ordered_classes.values()),i+1, frameon=False)
            ax.imshow(selection[elem],interpolation='nearest')
            ax.axis('off')
    else: 
        plt.xticks(tick_marks, ordered_classes.values(), rotation=45, fontsize=font_size)
        
    plt.figure(1)
    plt.savefig('conf_mat')

In [21]:
data_array_names = [class_dict[elem] for elem in data_array_label]
prediction_heat_names = [class_dict[elem] for elem in prediction_heat]

cnf_matrix = confusion_matrix(data_array_names, prediction_heat_names, labels=list(class_dict.values()))
cnf_matrix[np.isnan(cnf_matrix)] = 0

plot_confusion_matrix(cnf_matrix, class_dict=class_dict, title='Confusion matrix',normalize=True)
print(classification_report(data_array_names, prediction_heat_names))

In [23]:
data_array_names = [class_dict[elem] for elem in data_array_label]
prediction_heat_names = [class_dict[elem] for elem in prediction_heat]

cnf_matrix = confusion_matrix(data_array_names, prediction_heat_names, labels=list(class_dict.values()))
cnf_matrix[np.isnan(cnf_matrix)] = 0

plot_confusion_matrix(cnf_matrix, class_dict=class_dict, title='Confusion matrix',normalize=True)
print(classification_report(data_array_names, prediction_heat_names))

In [290]:
data_array_names_simple, prediction_heat_names_simple, class_dict_simple = simplify_labels(data_array_names, prediction_heat_names, res)

defaultdict(<class 'int'>, {1: 7, 2: 7, 3: 10, 4: 5})


In [295]:
# index of image to select for each class
selection = {
    'Extremely Rare < 50': 102, 
    'No Animal' : 10, 
    'Normal > 1000': 30, 
    'Rare > 200': 1,
    'Scarce > 500': 10, 
    'Very Rare > 50': 10
}

def get_img_from_selection(selection=selection):
    image_selection = {}
    for i, cat in enumerate(np.unique(data_array_names_simple)):
        idx_animal = (data_array_names_simple == cat).nonzero()[0]
        data_generator.batch_index = idx_animal[selection[cat]]
        image = data_generator.next()[0][0]
    #     plt.title(cat + "    " + str(idx_animal[selection[cat]]))
        image_selection[cat] = image
    #     plt.imshow(image)
    #     plt.show()
    #     plt.close()
    return image_selection

In [24]:
cnf_matrix = confusion_matrix(data_array_names_simple, prediction_heat_names_simple, labels=list(class_dict_simple.values()))
cnf_matrix[np.isnan(cnf_matrix)] = 0
np.set_printoptions(precision=2)

plot_confusion_matrix(cnf_matrix, class_dict=class_dict_simple, title='Confusion matrix',
                      normalize=True, selection=None)
print(classification_report(data_array_names_simple, prediction_heat_names_simple))

In [12]:
train_path = 'D://Species//train_cropped'

data_generator_train = test_datagen.flow_from_directory(
    train_path,
    target_size=(HEIGHT, WIDTH),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)
train_names = [class_dict[elem] for elem in data_generator_train.classes]

Found 408979 images belonging to 87 classes.


In [16]:
train_names_simple, _, _ = simplify_labels(train_names, train_names, get_data_dist(train_path))
res_train = np.unique(train_names_simple, return_counts=True)
res_train = dict(zip(res_train[0],res_train[1]))

In [25]:
sorted_res = collections.OrderedDict(sorted(res_train.items()))

size = len(sorted_res.keys())

plt.figure(0, figsize=(15,20))
plt.barh(list(sorted_res.keys()), list(sorted_res.values()))
plt.xticks(rotation='vertical', size='xx-large')
# plt.vlines(200, -1, size + 1, label='split at 200')
# plt.vlines(100, -1, size + 1 , label='split at 100')
# plt.vlines(50, -1, size + 1, label='split at 50')
# plt.ylim(ymax=1000)
# plt.legend()

sorted_res_value = collections.OrderedDict(sorted(res_train.items(), key=lambda kv: kv[1]))

plt.figure(1, figsize=(15,20))
plt.barh(list(sorted_res_value.keys()), list(sorted_res_value.values()))
plt.xticks(rotation='vertical', fontsize=30)
plt.yticks(list(sorted_res_value.keys()), fontsize=30)
# plt.xlim(xmax=1000)
# plt.vlines(200, -1, size + 1, label='split at 200')
# plt.vlines(100, -1, size + 1, label='split at 100')
# plt.vlines(50, -1, size + 1, label='split at 50')
# plt.legend()

In [22]:
def train_test_split(path, split=.2):
        for subdir, dirs, files in os.walk(path):
            if dirs == []:
                test_size = math.ceil(len(files) * split)
                idx = np.array(random.sample(range(0, len(files)), test_size))
                files = np.array(files)
                subdir_new = str(subdir).replace('\\cropped\\','\\test_cropped\\')
                print(subdir_new)
                if not (os.path.exists(subdir_new) and os.path.isdir(subdir_new)):
                    os.makedirs(subdir_new)
                for elem in files[idx]:
                    file = subdir + '\\' + elem
                    move(file, subdir_new + '\\' + elem)

In [23]:
# 'U:\\PycharmProjects\\AnimalBiometrics\\data\\Clouded_leopard_ID\\train_cleaned_copy'
# train_test_split("D:\Species\cropped")

D:\Species\test_cropped\Asian small-clawed otter
D:\Species\test_cropped\Banded civet
D:\Species\test_cropped\Banded linsang
D:\Species\test_cropped\Banteng
D:\Species\test_cropped\Bay cat
D:\Species\test_cropped\Bearded pig
D:\Species\test_cropped\Binturong
D:\Species\test_cropped\Bird
D:\Species\test_cropped\Black-capped babbler
D:\Species\test_cropped\Black-crowned pitta
D:\Species\test_cropped\Blue flycather
D:\Species\test_cropped\Blue-headed pitta
D:\Species\test_cropped\Bornean ground-cuckoo
D:\Species\test_cropped\Bornean necklaced partridge
D:\Species\test_cropped\Bornean peacock pheasant
D:\Species\test_cropped\Bornean yellow muntjac
D:\Species\test_cropped\Buff-vented bulbul
D:\Species\test_cropped\Bushy fish owl
D:\Species\test_cropped\Changeable hawk eagle
D:\Species\test_cropped\Clouded leopard
D:\Species\test_cropped\Collared mongoose
D:\Species\test_cropped\Common palm civet
D:\Species\test_cropped\Common porcupine
D:\Species\test_cropped\Crested fireback
D:\Species\tes

In [54]:
def crop_images(crop_top=0, crop_bottom=0, path='U:\\PycharmProjects\\AnimalBiometrics\\data\\Clouded_leopard_ID\\train_cleaned_copy'):
    for subdir, dirs, files in os.walk(path):
        print(dirs)
        if dirs == []:
            files = np.array(files)
            subdir_new = str(subdir).replace('\\raw\\','\\train_cropped\\')
            if not (os.path.exists(subdir_new) and os.path.isdir(subdir_new)):
                os.makedirs(subdir_new)
            for elem in files:
                if elem == 'Thumbs.db': continue
                img = Image.open(subdir + '\\' + elem)
                width, height = img.size
                cropped_img = img.crop((0, 0+crop_top, width, height-crop_bottom))
#                 cropped_img.show()
                cropped_img.save(subdir_new + '\\' + elem)
    
    

In [26]:
# crop_images(30,65,"D:/Species")

In [10]:
train_df = pd.read_csv('U:/PycharmProjects/AnimalBiometrics/data/annotations/Training_boxes_animal_biometrics.csv', header=0, usecols=["Label",'External ID','Dataset Name'])
test_df = pd.read_csv('U:/PycharmProjects/AnimalBiometrics/data/annotations/Test_boxes_animal_biometrics.csv', header=0, usecols=["Label",'External ID','Dataset Name'])

In [19]:
train_df_copy = train_df.copy()
test_df_copy = test_df.copy()
threshold = 4

def convert_to_retina_net_csv(df):
    train_out = []

    for i, data_point in enumerate(df.values):
        bbox, img_id, label = data_point
        
        im_path = 'U:/PycharmProjects/AnimalBiometrics/data/Clouded_leopard_ID/train_cleaned_cropped/{0}/'.format(label)
        
        if sum((len(list(filter(lambda a: a != 'Thumbs.db', f))) for _, _, f in os.walk(im_path))) < threshold: continue
        
        if bbox == 'Skip':
            x_min, x_max, y_min, y_max = None, None, None, None
        else:
            bbox = json.loads(bbox)
            coordinates = bbox['Animal'][0]['geometry']
#             print(coordinates)
            ys = [coordinates[i]['y'] for i in range(4)]
            xs = [coordinates[i]['x'] for i in range(4)]

        entry = [os.path.join(im_path, img_id), np.min(xs), np.min(ys), np.max(xs), np.max(ys), label]
        train_out.append(entry)

    bbox_df = pd.DataFrame(data=train_out, dtype=np.int32)
    return bbox_df

In [20]:
convert_to_retina_net_csv(train_df_copy).to_csv('./data/annotations/labeled_bboxes_train_RetinaNet_min_4.csv', index=False, header=False)

In [79]:
def create_class_mapping(path='U:\\PycharmProjects\\AnimalBiometrics\\data\\Clouded_leopard_ID\\train_cleaned_copy'):
    for subdir, dirs, files in os.walk(path):
        if dirs == []: break
        class_name_mapping_df = pd.DataFrame(list(zip(dirs,range(len(dirs)))))
        class_name_mapping_df.columns = ['class_name','id']
        class_name_mapping_df.to_csv('./data/annotations/class_mapping.csv', index=False)

In [80]:
# create_class_mapping()

In [6]:
def get_members(zip):
    parts = []
    # get all the path prefixes
    for name in zip.namelist():
        # only check files (not directories)
        if not name.endswith('/'):
            # keep list of path elements (minus filename)
            parts.append(name.split('/')[:-1])
    # now find the common path prefix (if any)
    prefix = os.path.commonprefix(parts)
    if prefix:
        # re-join the path elements
        prefix = '/'.join(prefix) + '/'
    # get the length of the common prefix
    offset = len(prefix)
    # now re-set the filenames
    for zipinfo in zip.infolist():
        name = zipinfo.filename
        # only check files (not directories)
        if len(name) > offset:
            # remove the common prefix
            zipinfo.filename = name[offset:]
            yield zipinfo    

In [5]:
path = "D:/Species"
_zip = zipfile.ZipFile(os.path.join(path, 'sdc01.zip'),"r")
_zip.extractall(path, get_members(_zip))

In [4]:
import zipfile

def extract_zips(path='D:/Species'):
    zips = [name for name in os.listdir(path) if zipfile.is_zipfile(os.path.join(path, name)) ]

    for file in zips:
        print(file)
        temp = zipfile.ZipFile(os.path.join(path, file),"r")
        temp.extractall(path, get_members(temp))
    

In [7]:
extract_zips()

sdc01.zip
sdc02.zip
sdc03.zip
sdc04.zip
sdc05.zip
sdc06.zip
sdc07.zip
sdc08.zip
sdc09.zip
sdc11.zip
sdc12.zip
sdc13.zip
sdc14.zip
sdc15.zip
sdc16.zip
sdc17.zip
sdc18.zip
sdc19.zip
sdc20.zip
sdc21.zip
sdc22.zip
sdc23.zip
sdc24.zip
sdc25.zip
sdc26.zip
sdc27.zip
sdc28.zip
sdc29.zip
sdc30.zip
sdc31.zip
sdc32.zip
sdc33.zip
sdc34.zip
sdc35.zip
sdc36.zip
sdc37.zip
sdc38.zip
sdc39.zip
sdc40.zip
sdc41.zip
sdc42.zip
sdc43.zip
sdc44.zip
sdc45.zip
sdc46.zip
sdc47.zip
sdc48.zip
sdc49.zip
sdc50.zip
sdc51.zip
sdc52.zip
sdc53.zip
sdc54.zip
sdc55.zip
sdc56.zip
sdc57.zip
sdc58.zip
sdc59.zip
sdc60.zip
sdc61.zip
sdc62.zip
sdc63.zip
sdc64.zip
SDF01.zip
SDF02.zip
SDF03.zip
SDF04.zip
SDF05.zip
SDF06.zip
SDF07.zip
SDF08.zip
SDF09.zip
SDF10.zip
SDF11.zip
SDF12.zip
SDF13.zip
SDF14.zip
SDF15.zip
SDF16.zip
SDF17.zip
SDF18.zip
SDF19.zip
SDF20.zip
SDF21.zip
SDF22.zip
SDF24.zip
SDF25.zip
SDF26.zip
SDF27.zip
SDF28.zip
SDF29.zip
SDF30.zip
SDF31.zip
SDF32.zip
SDF33.zip
SDF34.zip
SDF35.zip
SDF36.zip
SDF37.zip
SDF38.zip


In [20]:
def crop_images_root(crop_top=0, crop_bottom=0, path='U:\\PycharmProjects\\AnimalBiometrics\\data\\Clouded_leopard_ID\\train_cleaned_copy'):
    for subdir, dirs, files in os.walk(path):
        print(subdir, 'cropped' not in subdir)
        if dirs == [] and 'cropped' not in subdir:
            files = np.array(files)
            split = str(subdir).split("\\")
            subdir_new = os.path.join(split[0],"cropped",split[1])
            if not (os.path.exists(subdir_new) and os.path.isdir(subdir_new)):
                os.makedirs(subdir_new)
            for elem in files:
                img_path_new = subdir_new + '\\' + elem
#                 print(os.path.exists(img_path_new))
                if elem == 'Thumbs.db' or os.path.exists(img_path_new): continue
                try:
                    img = Image.open(subdir + '\\' + elem)
                    width, height = img.size
                    cropped_img = img.crop((0, 0+crop_top, width, height-crop_bottom))
    #                 cropped_img.show()
                    cropped_img.save(subdir_new + '\\' + elem)
                except: 
                    print(subdir + '\\' + elem)

In [21]:
crop_images_root(30,65,"D:/Species")

D:/Species True
D:/Species\Asian small-clawed otter True
D:/Species\Banded civet True
D:/Species\Banded linsang True
D:/Species\Banteng True
D:/Species\Bay cat True
D:/Species\Bearded pig True
D:/Species\Bearded pig\Sambar deer True
D:/Species\Binturong True
D:/Species\Bird True
D:/Species\Black-capped babbler True
D:/Species\Black-crowned pitta True
D:/Species\Blue flycather True
D:/Species\Blue-headed pitta True
D:/Species\Bornean ground-cuckoo True
D:/Species\Bornean necklaced partridge True
D:/Species\Bornean peacock pheasant True
D:/Species\Bornean yellow muntjac True
D:/Species\Buff-vented bulbul True
D:/Species\Bushy fish owl True
D:/Species\Changeable hawk eagle True
D:/Species\Clouded leopard True
D:/Species\Collared mongoose True
D:/Species\Common palm civet True
D:/Species\Common porcupine True
D:/Species\Crested fireback True
D:/Species\Crested serpent eagle True
D:/Species\cropped False
D:/Species\cropped\Asian small-clawed otter False
D:/Species\cropped\Banded civet False

In [11]:
from vis.visualization import visualize_activation

def print_comparison(indices, images, predictions, labels, trained_model, activation_layer='predictions'):
    
    trained_model.layers[utils.find_layer_idx(trained_model, "predictions")].activation = keras.activations.linear
    
    for i, img_number in enumerate(indices):
        for j in range(50, 98):
            activation_layer='activation_{}'.format(j)
            fig = plt.figure(i, figsize=(20,10))
            ax1 = fig.add_subplot(121)
    #         print(images)
            ax1.set_title(class_dict[labels[img_number]])
            ax1.imshow(images[img_number])
            activation = visualize_activation(n_model, utils.find_layer_idx(trained_model, activation_layer),
                                       seed_input=images[img_number],
                                       filter_indices=[labels[img_number]],
    #                                    filter_indices=None,
    #                                        penultimate_layer_idx=utils.find_layer_idx(trained_model, activation_layer),
    #                                    penultimate_layer_idx=None,
                                       backprop_modifier=None, 
                                       grad_modifier=None)

            ax2 = fig.add_subplot(122)
            ax2.set_title(class_dict[predictions[img_number]])
            ax2.imshow(activation)
            plt.show()

In [28]:
print_comparison(wrong_pred, data_array, prediction_heat, data_array_label, n_model)

In [27]:
print_comparison(correct_pred, data_array, prediction_heat, data_array_label, n_model)